In [ ]:
# using Pkg
# Pkg.activate("../../Radial Duality/MultiCenterProject/MCRD_env")

In [ ]:
using LinearAlgebra
using Random, Distributions
using Printf
using Plots
using LaTeXStrings
Random.seed!(1234567) # Setting the seed

In [ ]:
# import MRM module
include("MRMs.jl")

In [ ]:
using JuMP
using MosekTools
using MathOptInterface

In [ ]:
function solveQCQPWithMosek(Ps, qs, rs, m, n)
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    set_attribute(model, "INTPNT_CO_TOL_DFEAS", 1e-10)

    @variable(model, xvar[1:n]);

    for j in 2:m+1
        @constraint(model, rs[j] - qs[j]'*xvar - 0.5*xvar'*Ps[j]*xvar >= 0);
    end

    @objective(model, Max, rs[1] - qs[1]'*xvar - 0.5*xvar'*Ps[1]*xvar)
    optimize!(model)

    mosek_sol = JuMP.value.(xvar)
    mosek_obj_value = JuMP.objective_value(model)
    status = JuMP.termination_status(model)
    return mosek_sol, mosek_obj_value, status
end

# gauges for mosek solution
function getFeasVal(sol, qcqp, m)
    gauge_vals = Vector{Float64}(undef, m);
    for i in 1:m gauge_vals[i] = qcqp.identifiers[i](sol, false)[1] end;
    feasibility = maximum(gauge_vals)
    return feasibility
end;

In [ ]:
function quadratic(x, P, q, r)
    return r - q'*x - 0.5*x'*P*x
end

### Run time experiment

In [ ]:
# problem params
tol = 10^(-9)
n = 200
lambda = 0.01
nu = 0.1
sigma = 10.0
x_sl = zeros(n)
x0 = x_sl;

In [ ]:
# pre-allocate dictionaries for storing values
m_vals = [10, 100, 1000]
K = length(m_vals)
mats = Dict{Int64, Array{Matrix{Float64}}}()
vecs = Dict{Int64, Array{Vector{Float64}}}()
cons = Dict{Int64, Array{Float64}}()
center_vecs = Dict{Int64, Array{Vector{Float64}}}()
f_x0s = Dict{Int64, Float64}()
tau0s = Dict{Int64, Float64}()

# radial set-up objects
params = Dict{Int64, Vector{MRM.quadFormConsGaugeParams}}()


# mosek stuff
mosek_sols = Dict{Int64, Vector{Float64}}()
mosek_vals = Dict{Int64, Float64}()

# dictionaries for each method
subgrad_ret = Dict{Int64, Dict}()
smooth_ret = Dict{Int64, Dict}()
genGrad_ret = Dict{Int64, Dict}();

In [ ]:
for k = 1:K
    m = m_vals[k]
    Ps, qs, rs, best_centers = MRM.sampleQCQP(n, m, lambda, sigma, nu)
    mats[m], vecs[m], cons[m], center_vecs[m] = Ps, qs, rs, best_centers
    f_x0 = quadratic(x0, Ps[1], qs[1], rs[1])
    f_x0s[m] = f_x0
    tau0s[m] = 1 / f_x0
    rad_params = Vector{MRM.quadFormConsGaugeParams}(undef, m+1)
    for j=1:m+1
        rad_params[j] = MRM.quadFormConsGaugeParamsInitializer(Ps[j], qs[j], rs[j], best_centers[j])
    end
    params[m] = rad_params
end

In [ ]:
# solve with Mosek
for k = 1:K
    m = m_vals[k]
    Ps, qs, rs, best_centers = mats[m], vecs[m], cons[m], center_vecs[m]
    mosek_sols[m], mosek_vals[m], _ = solveQCQPWithMosek(Ps, qs, rs, m, n)
end

In [ ]:
# parallel MRM parameters
N = 16
T_vals = [500, 400, 100]
rho_tilde = 1.0
tot_time = 3000.0
b = 4.0;

In [ ]:
function getFomValues(fom_iter_pair, rho_tilde, m, mats, vecs, cons, params)
    fom, T = fom_iter_pair
    Ps, qs, rs = mats[m], vecs[m], cons[m]
    identifier_oracle = MRM.quadFormConsHuberGaugeOracle
    if fom == MRM.subgradientMethod
        identifier_oracle = MRM.quadFormConsGaugeOracle
    elseif fom == MRM.fastGenGradMethod
        identifier_oracle = MRM.quadFormConsGaugeSquareOracle
    end
    inst = MRM.instanceConstructor(Ps[1], qs[1], rs[1], params[m], identifier_oracle, tol)
    ret = Dict{String, Array{Float64}}()
    tau0 = tau0s[m]
    mrm_ret = MRM.parallelMRM(fom,inst,(x0,tau0),T,b=b,N=N,tol=tol,rho_tilde=rho_tilde,tot_time=tot_time)
    ret["sols"] = mrm_ret[1].iterate
    ret["vals"] = mrm_ret[2]
    ret["process_vals"] = mrm_ret[3]
    ret["process_times"] = mrm_ret[5]
    
    return ret
end

In [ ]:
for k = 1:K
    m = m_vals[k]
    time_now = time()
    subgrad_pair = (MRM.subgradientMethod, 100*T_vals[k])
    subgrad_ret[m] = getFomValues(subgrad_pair, rho_tilde, m, mats, vecs, cons, params)
    time_sub = (time() - time_now) / 60
    @printf("The total time for subgrad is %.4f minutes. \n", time_sub)
end

In [ ]:
using HDF5, JLD

In [ ]:
save("../Plots and Data/subgrad_ret_data.jld", "data", subgrad_ret)

In [ ]:
for k = 1:K
    m = m_vals[k]
    time_now = time()
    smooth_pair = (MRM.fastSmoothingMethod, 100*T_vals[k])
    smooth_ret[m] = getFomValues(smooth_pair, rho_tilde, m, mats, vecs, cons, params)
    time_smooth = (time() - time_now) / 60
    @printf("The total time for smooth is %.4f minutes. \n", time_smooth)
end

In [ ]:
save("../Plots and Data/smooth_ret_data.jld", "data", smooth_ret)

In [ ]:
for k = 1:K
    m = m_vals[k]
    time_now = time()
    genGrad_pair = (MRM.fastGenGradMethod, 4*T_vals[k])
    genGrad_ret[m] = getFomValues(genGrad_pair, rho_tilde, m, mats, vecs, cons, params)
    time_gen = (time() - time_now) / 60
    @printf("The total time for gengrad is %.4f minutes. \n", time_gen)
end

In [ ]:
save("../Plots and Data/genGrad_ret_data.jld", "data", genGrad_ret)

### Plots

In [ ]:
labels = [latexstring("\$\\mathtt{subgrad}\$"), 
          latexstring("\$\\mathtt{smooth}\$"), 
          latexstring("\$\\mathtt{genGrad}\$")];

In [ ]:
# make iterations plot Data
function makeIterPlotData(m, mosek_vals, fom_ret, key="vals")
    mosek_val = mosek_vals[m]
    best_seen = fom_ret[m][key]
    if key == "vals"
        init_gap = mosek_val - best_seen[1]
    elseif key == "process_vals"
        init_gap = mosek_val - best_seen[1,1]
    end
    gaps = mosek_val .- best_seen
    y = gaps / init_gap
    return y
end

# make time plot data
function makeTimePlotData(m, mosek_vals, fom_ret)
    mosek_val = mosek_vals[m]
    best_seen, times = fom_ret[m]["vals"], fom_ret[m]["process_times"]
    xbest = cumsum(sum(times, dims=1), dims=2)[1, :]
    gaps = mosek_val .- best_seen
    init_gap = gaps[1]
    ybest = gaps / init_gap
    return xbest, ybest
end

In [ ]:
# make iteration plots
# initialize plot
p = plot()
k1= 1
k2= K
lw = 2
fs = 12
for k = k1:k2
    m = m_vals[k]
    x_upper = length(smooth_ret[m]["vals"])
    y_sub = makeIterPlotData(m, mosek_vals, subgrad_ret, "vals")
    y_sm = makeIterPlotData(m, mosek_vals, smooth_ret, "vals")
    y_gen = makeIterPlotData(m, mosek_vals, genGrad_ret, "vals")
    p = plot(y_sub, linewidth=lw, label=labels[1], xlims=(1, x_upper), legend=:bottomleft)
    plot!(y_sm, label=labels[2], linewidth=lw)
    plot!(y_gen, label=labels[3], linewidth=lw)
    plot!(xaxis=:log, yaxis=:log, xlabel="Iterations", ylabel="Relative Objective Gap")
    ylims = ylims=(10.0^(-9), 1)
    yticks = [10.0^(-9+j) for j=1:9]
    plot!(xticks=[10^j for j=1:4], yticks=yticks, ylims=ylims)
    plot!(xtickfontsize=fs-2, ytickfontsize=fs, xformatter=:auto)
    plot!(guidefontsize=fs, legendfontsize=fs)
    savefig("../Plots and Data/accu_vs_iter_m=$(m_vals[k]).pdf")
end
plot(p)

In [ ]:
# Make time plots
# initialize plot
p = plot()
k1= 1
k2= K
lw = 2
fs = 14
for k = k1:k2
    m = m_vals[k]
    time_lim = 3*10.0^(k)
    x_sub, y_sub = makeTimePlotData(m, mosek_vals, subgrad_ret)
    x_sm, y_sm = makeTimePlotData(m, mosek_vals, smooth_ret)
    x_gen, y_gen = makeTimePlotData(m, mosek_vals, genGrad_ret)
    xlims=(tol, time_lim)
    p = plot(x_sub, y_sub, label=labels[1], lw=2, yaxis=:log, xlims=xlims,legend=:right)
    plot!(x_sm, y_sm, label=labels[2], lw=2)
    plot!(x_gen, y_gen, label=labels[3], lw=2)
    plot!(ylabel="Relative Objective Gap", xlabel="Time (s)")
    ylims = ylims=(10.0^(-9), 1)
    yticks = [10.0^(-9+j) for j=1:9]
    xticks = [5*10^(k-1)*j for j=1:5]
    plot!(xticks=xticks, yticks=yticks, ylims=ylims)
    plot!(xtickfontsize=fs-2, ytickfontsize=fs-2, xformatter=:auto)
    plot!(guidefontsize=fs, legendfontsize=fs)
    savefig("../Plots and Data/accu_vs_time_m=$(m_vals[k]).pdf")
end
plot(p)

In [ ]:
# subgrad_ret = load("../Plots and Data/subgrad_ret_data.jld")["data"]
# smooth_ret = load("../Plots and Data/smooth_ret_data.jld")["data"]
# genGrad_ret = load("../Plots and Data/genGrad_ret_data.jld")["data"]